In [3]:
!pip install theano

    100% |████████████████████████████████| 2.8MB 7.8MB/s 
  Running setup.py bdist_wheel for theano ... - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/87/1e/28/4a63195927452fb42d4ea6d1e5b3b1690409d66802cc9e1e6e
Successfully built theano


Using Theano it is possible to attain speeds rivaling hand-crafted C implementations for problems involving large amounts of data

In [0]:
import numpy
import theano
from theano import tensor

a = tensor.dscalar()
b = tensor.dscalar()
c = a + b

f = theano.function([a,b], c)

# bind 1.5 to 'a', 2.5 to 'b', and evaluate 'c'
assert 4.0 == f(1.5, 2.5)

Now that we’ve created our function we can use it:

In [9]:
numpy.allclose(f(16.3, 12.1), 28.4)

True

In [5]:
from theano import pp
print(pp(c))

numpy.allclose(c.eval({a : 16.3, b : 12.1}), 28.4)


(<TensorType(float64, scalar)> + <TensorType(float64, scalar)>)


True

eval() will be slow the first time you call it on a variable – it needs to call function() to compile the expression behind the scenes. Subsequent calls to eval() on that same variable will be fast, because the variable caches the compiled function.

In [0]:
x = tensor.dmatrix('x')
y = tensor.dmatrix('y')
z = x + y
f = theano.function([x, y], z)

dmatrix is the Type for matrices of doubles. Then we can use our new function on 2D arrays or numpy arrays


In [12]:
f([[1, 2], [3, 4]], [[10, 20], [30, 40]])
f(numpy.array([[1, 2], [3, 4]]), numpy.array([[10, 20], [30, 40]]))

array([[11., 22.],
       [33., 44.]])




1. bscalar, bvector, bmatrix, brow, bcol - can be used to store as bytes
2. wscalar, wmatrix, ..... - can be used for 16-bit integers
3. iscalar, imatrix, ...... - can be used for 32-bit integers
4. lscalar, lmatrix, ...... - can be used for 64-bit integers
5. fscalar, fmatrix, .... - can be used for float
6. dscalar, dmatrix, .... - can be used for double
7. *cscalar*, cmatrix,.... - can be used for complex kind of variables






In [13]:
a = theano.tensor.vector() 
out = a + a ** 10              
f = theano.function([a], out)  
print(f([0, 1, 2]))

[   0.    2. 1026.]


In [14]:
b = theano.tensor.vector()
out = a ** 2 + b ** 2 + 2 * a * b
f1 = theano.function([a, b],out)
print(f1([1, 2], [4, 5]))


[25. 49.]


In [15]:
x = theano.tensor.dmatrix('x')
s = 1 / (1 + theano.tensor.exp(-x))
logistic = theano.function([x], s)
logistic([[0, 1], [-1, -2]])

s2 = (1 + theano.tensor.tanh(x / 2)) / 2
logistic2 = theano.function([x], s2)
logistic2([[0, 1], [-1, -2]])

array([[0.5       , 0.73105858],
       [0.26894142, 0.11920292]])

Can also perform multiple functions on particular data 

In [16]:
a, b = theano.tensor.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
prod = a*b
f = theano.function([a, b], [diff, abs_diff, diff_squared])
f([[1, 1], [1, 1]], [[0, 1], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[1., 0.],
        [1., 2.]]), array([[1., 0.],
        [1., 4.]])]

In [17]:
x, y = theano.tensor.dscalars('x', 'y')
z = x + y
f = theano.function([x, theano.In(y, value=1)], z)
f(33)
f(33,8)


array(41.)

Can also add default values 

In [18]:
w = theano.tensor.dscalars('w')
z = (x + y) * w
f = theano.function([x, theano.In(y, value=1), theano.In(w, value=2, name='w_by_name')], z)
f(33, w_by_name=1, y=0)

array(33.)

In [19]:
f(33, 2)

array(70.)

Can also include shared variables whose state can be updated whenever the function is called

In [0]:
from theano import shared
state = shared(0)
inc = theano.tensor.iscalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)])

In [21]:
accumulator(5)

array(0)

In [22]:
print(state.get_value())

5


To reset the state 

In [0]:
state.set_value(-1)

In [24]:
accumulator(5)

array(-1)

In [25]:
 decrementor = theano.function([inc], state, updates=[(state, state-inc)]) #can use the same shared variable in different functions
 decrementor(2)

array(4)

If you donot want to use the shared variable in a function, you can replace it with a different given value

In [26]:
fn_of_state = state * 2 + inc
foo = theano.tensor.scalar(dtype=state.dtype)
skip_shared = theano.function([inc, foo], fn_of_state, givens=[(state, foo)])
skip_shared(1, 3)


array(7)

In [27]:
print(state.get_value())  #doesnot change previous state value

2


In [28]:
new_state = theano.shared(0)
new_accumulator = accumulator.copy(swap={state:new_state})
new_accumulator(100)
print(new_state.get_value())

100


In [29]:
print(state.get_value()) #old functions state value does not change 

2


In [30]:
null_accumulator = accumulator.copy(delete_updates=True)  #can also avoid updates 
null_accumulator(9000)

[array(2)]

In [31]:
print(state.get_value())

2


For random variables - Theano will allocate a NumPy RandomStream object (a random number generator) for each such variable, and draw from it as necessary

In [0]:
from theano.tensor.shared_randomstreams import RandomStreams
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2,2))   #a random uniform distribution of 2 * 2 dimensions (these only work on the CPU)
rv_n = srng.normal((2,2))    #a random normal distribution of 2 * 2 dimensions (these only work on the CPU)
f = theano.function([], rv_u)
g = theano.function([], rv_n, no_default_updates=True)    #Not updating rv_n.rng
nearly_zeros = theano.function([], rv_u + rv_u - 2 * rv_u)
f_val0 = f()
f_val1 = f()
g_val0 = g()
g_val1 = g()


In [33]:
f_val0



array([[0.12672381, 0.97091597],
       [0.13989098, 0.88754825]])

In [34]:
f_val1 #we are getting different number each time the function is called because the internal state of the random number generator is automatically updated

array([[0.31971415, 0.47584377],
       [0.24129163, 0.42046081]])

In [35]:
g_val0 

array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]])

In [36]:
g_val1  #as can be observed, calling g() multiple times wil not give different values as no_default_updates has been set to True

array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]])

Seed in a random number generator is a given number from which the random number generation starts. You can seed **just one random variable**  or **all the random stram variables** by seeding or assigning to the .rng attribute, using .rng.set_value(). 

In [0]:
rng_val = rv_u.rng.get_value(borrow=True)   # Get the specific rng for rv_u
rng_val.seed(89234)                         # seeds the generator
rv_u.rng.set_value(rng_val, borrow=True)

srng.seed(902340)  # seeds rv_u and rv_n with different seeds each


An example of how “random states” can be transferred from one theano function to another is shown below.

In [0]:
from __future__ import print_function
from theano.sandbox.rng_mrg import MRG_RandomStreams
from theano.tensor.shared_randomstreams import RandomStreams

In [0]:
class Graph():
...     def __init__(self, seed=123):
...         self.rng = RandomStreams(seed)
...         self.y = self.rng.uniform(size=(1,))
g1 = Graph(seed=123)
f1 = theano.function([], g1.y)


In [40]:
g2 = Graph(seed=987)
f2 = theano.function([], g2.y)
print(f1())
print(f2())

[0.72803009]
[0.55056769]


In [41]:
def copy_random_state(g1, g2):
...     if isinstance(g1.rng, MRG_RandomStreams):
...         g2.rng.rstate = g1.rng.rstate
...     for (su1, su2) in zip(g1.rng.state_updates, g2.rng.state_updates):
...         su2[0].set_value(su1[0].get_value())
copy_random_state(g1, g2)
print(f1())
print(f2())

[0.59044123]
[0.59044123]



MRG_RandomStreams - runs on cpu and gpu while RandomStreams works only on cpu

Practical Logistic Regression Example


In [6]:
import theano.tensor as T
rng = numpy.random

N = 400                                   # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.dmatrix("x")
y = T.dvector("y")

# initialize the weight vector w randomly
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
print(w.get_value())
print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                         

# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
target = D[1]
predicted = predict(D[0])
predicted = predicted.astype(int)
count = 0
for i in range(len(target)):
  if (target[i] == predicted[i] ):
    count = count+1

print("Accuracy")
print(count/len(target))


Initial model:
[ 5.47475471e-01 -8.80551291e-01  8.73338800e-01  1.33629194e+00
 -1.61163453e+00  8.63800965e-01 -1.77181291e+00 -2.41469657e-01
 -2.52920578e+00  1.61897169e+00  9.35475352e-01 -4.86689605e-01
 -5.98861399e-01 -1.72415864e-01 -1.49231053e-01 -2.00365537e-02
  2.17817433e-01 -1.77510576e-01 -7.75872286e-01  3.51690704e+00
 -5.52388567e-01  1.21468904e+00 -2.17096988e+00  7.29555778e-01
  7.69032130e-01 -1.78975504e+00 -9.02278944e-01 -1.65761681e+00
  7.02574366e-01 -2.25919399e-01 -7.35823140e-01  9.08827011e-01
 -9.52143947e-01 -1.15169871e+00  7.18310334e-02 -3.14807904e-02
  8.33142203e-01 -3.16685769e-01  4.65460633e-01  5.17492074e-01
  9.74028430e-02 -7.13493768e-01  7.26484569e-01  8.90105571e-01
 -9.36098103e-01 -4.75430091e-01  9.85458112e-01 -9.17367555e-01
  6.78039469e-01 -6.10388767e-01 -2.82742542e+00 -1.04841807e+00
  1.17224033e+00 -2.22612245e-01 -4.69459578e-01 -3.78755629e-01
 -1.44371711e+00 -7.96233039e-01  1.37678104e+00  1.62461028e+00
 -1.711121

Derivatives can be described just as easily

In [7]:
x = T.dscalar('x')
y = x ** 2
gy = T.grad(y, x)
pp(gy)
f = theano.function([x], gy)
f(4) # - here the derivative will just be 2*x 

array(8.)

In [67]:
pp(f.maker.fgraph.outputs[0]) # as we can see the optimiser simplified the function to 2 * x

'(TensorConstant{2.0} * x)'

Another Example 

In [9]:
x = T.dmatrix('x')
s = T.sum(1 / (1 + T.exp(-x)))
gs = T.grad(s, x) #The second argument to T.grad can also be a list and it gives a list of corresponding derivative values as an output
dlogistic = theano.function([x], gs)
dlogistic([[0, 1], [-1, -2]])

array([[0.25      , 0.19661193],
       [0.19661193, 0.10499359]])

Computing Jacobian 

In [10]:
x = T.dvector('x')
y = x ** 2
J, updates = theano.scan(lambda i, y, x : T.grad(y[i], x), sequences=T.arange(y.shape[0]), non_sequences=[y, x])
f = theano.function([x], J, updates=updates)
f([4, 4])

array([[8., 0.],
       [0., 8.]])

What we do in this code is to generate a sequence of ints from 0 to y.shape[0] using T.arange. Then we loop through this sequence, and at each step, we compute the gradient of element y[i] with respect to x. scan automatically concatenates all these rows, generating a matrix which corresponds to the Jacobian. However, there is also a direct function - theano.tensor.jacobian


Computing the Hessian matrix

In [11]:
x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy,x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = theano.function([x], H, updates=updates)
f([4, 4])

array([[2., 0.],
       [0., 2.]])

The R operator is built to evaluate the product between a Jacobian and a vector, namely \frac{\partial f(x)}{\partial x} v. In order to evaluate the R-operation of expression y, with respect to x, multiplying the Jacobian with v you need to do 

In [12]:
W = T.dmatrix('W')
V = T.dmatrix('V')
x = T.dvector('x')
y = T.dot(x, W)
JV = T.Rop(y, W, V)
f = theano.function([W, V, x], JV)
f([[1, 1], [1, 1]], [[2, 2], [2, 2]], [0,1])

array([2., 2.])

In similitude to the R-operator, the L-operator would compute a row vector times the Jacobian. The mathematical formula would be v \frac{\partial
f(x)}{\partial x}. 

In [15]:
W = T.dmatrix('W')
v = T.dvector('v')
x = T.dvector('x')
y = T.dot(x, W)
VJ = T.Lop(y, W, v)
f = theano.function([v,x], VJ)
f([2, 2], [0, 1])

array([[0., 0.],
       [2., 2.]])

Hessian times a Vector 

In [16]:
x = T.dvector('x')
v = T.dvector('v')
y = T.sum(x ** 2)
gy = T.grad(y, x)
vH = T.grad(T.sum(gy * v), x)
f = theano.function([x, v], vH)
f([4, 4], [2, 2]) #or R operator can be used too 

array([4., 4.])

Work is in progress to build a full hessian or jacobian complete matrix optimally 


If-Else and Switch 

In [17]:
from theano import tensor as T
from theano.ifelse import ifelse
import theano, time, numpy

a,b = T.scalars('a', 'b')
x,y = T.matrices('x', 'y')

z_switch = T.switch(T.lt(a, b), T.mean(x), T.mean(y))
z_lazy = ifelse(T.lt(a, b), T.mean(x), T.mean(y))

f_switch = theano.function([a, b, x, y], z_switch,
                           mode=theano.Mode(linker='vm'))
f_lazyifelse = theano.function([a, b, x, y], z_lazy,
                               mode=theano.Mode(linker='vm'))

val1 = 0.
val2 = 1.
big_mat1 = numpy.ones((10000, 1000))
big_mat2 = numpy.ones((10000, 1000))

n_times = 10

tic = time.clock()
for i in range(n_times):
    f_switch(val1, val2, big_mat1, big_mat2)
print('time spent evaluating both values %f sec' % (time.clock() - tic))

tic = time.clock()
for i in range(n_times):
    f_lazyifelse(val1, val2, big_mat1, big_mat2)
print('time spent evaluating one value %f sec' % (time.clock() - tic))

time spent evaluating both values 0.365559 sec
time spent evaluating one value 0.189599 sec


In this example, the IfElse op spends less time (about half as much) than Switch since it computes only one variable out of the two. This is because switch evaluates both the output variables, while ifelse evaluates only one output variable. 


Loops : scan 

Advantages of using scan over for loops:
Number of iterations to be part of the symbolic graph.
Minimizes GPU transfers (if GPU is involved).
Computes gradients through sequential steps.
Slightly faster than using a for loop in Python with a compiled Theano function.
Can lower the overall memory usage by detecting the actual amount of memory needed.

In [19]:
import numpy as np
# defining the tensor variables
X = T.matrix("X")
W = T.matrix("W")
b_sym = T.vector("b_sym")

results, updates = theano.scan(lambda v: T.tanh(T.dot(v, W) + b_sym), sequences=X)
compute_elementwise = theano.function(inputs=[X, W, b_sym], outputs=results)

# test values
x = np.eye(2, dtype=theano.config.floatX)
w = np.ones((2, 2), dtype=theano.config.floatX)
b = np.ones((2), dtype=theano.config.floatX)
b[1] = 2

print(compute_elementwise(x, w, b))

# comparison with numpy
print(np.tanh(x.dot(w) + b))

[[0.96402758 0.99505475]
 [0.96402758 0.99505475]]
[[0.96402758 0.99505475]
 [0.96402758 0.99505475]]


Computing the sequence x(t) = tanh(x(t - 1).dot(W) + y(t).dot(U) + p(T - t).dot(V))

In [20]:
X = T.vector("X")
W = T.matrix("W")
b_sym = T.vector("b_sym")
U = T.matrix("U")
Y = T.matrix("Y")
V = T.matrix("V")
P = T.matrix("P")

results, updates = theano.scan(lambda y, p, x_tm1: T.tanh(T.dot(x_tm1, W) + T.dot(y, U) + T.dot(p, V)),
          sequences=[Y, P[::-1]], outputs_info=[X])
compute_seq = theano.function(inputs=[X, W, Y, U, P, V], outputs=results)

# test values
x = np.zeros((2), dtype=theano.config.floatX)
x[1] = 1
w = np.ones((2, 2), dtype=theano.config.floatX)
y = np.ones((5, 2), dtype=theano.config.floatX)
y[0, :] = -3
u = np.ones((2, 2), dtype=theano.config.floatX)
p = np.ones((5, 2), dtype=theano.config.floatX)
p[0, :] = 3
v = np.ones((2, 2), dtype=theano.config.floatX)

print(compute_seq(x, w, y, u, p, v))

# comparison with numpy
x_res = np.zeros((5, 2), dtype=theano.config.floatX)
x_res[0] = np.tanh(x.dot(w) + y[0].dot(u) + p[4].dot(v))
for i in range(1, 5):
    x_res[i] = np.tanh(x_res[i - 1].dot(w) + y[i].dot(u) + p[4-i].dot(v))
print(x_res)

[[-0.99505475 -0.99505475]
 [ 0.96471973  0.96471973]
 [ 0.99998585  0.99998585]
 [ 0.99998771  0.99998771]
 [ 1.          1.        ]]
[[-0.99505475 -0.99505475]
 [ 0.96471973  0.96471973]
 [ 0.99998585  0.99998585]
 [ 0.99998771  0.99998771]
 [ 1.          1.        ]]


Currently, information regarding shape is used in two ways in Theano:

To generate faster C code for the 2d convolution on the CPU and the GPU, when the exact output shape is known in advance.

To remove computations in the graph when we only want to know the shape, but not the actual value of a variable. This is done with the Op.infer_shape method.

In [24]:
x = theano.tensor.matrix('x')
y = theano.tensor.matrix('y')
z = theano.tensor.join(0, x, y)
xv = numpy.random.rand(5, 4)
yv = numpy.random.rand(3, 3)
f = theano.function([x, y], z.shape)   #Does not throw error as it should 
theano.printing.debugprint(f)
f(xv,yv)

MakeVector{dtype='int64'} [id A] ''   4
 |Elemwise{Add}[(0, 0)] [id B] ''   3
 | |Shape_i{0} [id C] ''   2
 | | |x [id D]
 | |Shape_i{0} [id E] ''   1
 |   |y [id F]
 |Shape_i{1} [id G] ''   0
   |x [id D]


In [25]:
f = theano.function([x,y], z)# Do not take the shape.
theano.printing.debugprint(f)
f(xv,yv)

Join [id A] ''   0
 |TensorConstant{0} [id B]
 |x [id C]
 |y [id D]


ValueError: ignored